In [38]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
from procurement_lib import GoogleSheet, logging

In [39]:
paramss_csv=GoogleSheet("1Q8PtNX9Nj0Ha2sZwmW7fhp08aLN2Etw0hjU456bNiXA")

In [40]:
city_paramss = paramss_csv.get_as_dataframe("Info")

In [41]:
    query = """
    WITH 

    ful as (

        SELECT DISTINCT
            bo.order_id,
            ffg.close_date
            
        FROM postgres_broadleaf_federate."broadleaf.blc_order"                      bo
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg ON bfg.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo  ON fo.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop ON bop.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s   ON s.site_id = bo.site_disc
        LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot ON fot.fb_order_type_id=fo.fb_order_type_id
        LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment"     ba  ON ba.order_id = bo.order_id
        
        WHERE fo.fb_order_status_id IN (1,6,7,8)
            AND extract(year from ffg.close_date) = extract(year from CURRENT_DATE) 
            AND extract(month from ffg.close_date) = extract(month from CURRENT_DATE)
            AND extract(week from ffg.close_date) = extract(week from CURRENT_DATE)
            AND s.site_identifier_value IN ('BOG','SPO','MDE')
            AND bo.order_status = 'SUBMITTED'
            AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
            AND bop.archived = 'N'
            AND (fot.name IS NULL OR fot.name <> 'REFUND')
            
    ),

    all_data AS (

    SELECT 
        s.site_identifier_value as region_code,
        TO_CHAR(ful.close_date, 'WW') AS week,
        CASE
            WHEN (boida.adjustment_reason ILIKE '%Investment%')THEN 'ADDITIONAL INVESTMENT'
        ELSE 'OTHERS' END AS responsable,
        boida.adjustment_reason AS discount,
        bcat2.name as cat,
        bcat.name as subcat,
        bs.addl_product_id,
        bs2.name,
        CASE 
            WHEN bs.addl_product_id = 311459 THEN 0
            WHEN bs.addl_product_id = 194518 THEN 0.03
            WHEN bs.addl_product_id = 197660 THEN 0
            WHEN bs.addl_product_id = 280418 THEN 0.03
            WHEN bs.addl_product_id = 130229 THEN 0.03
            WHEN bs.addl_product_id = 413151 THEN 0
            WHEN bs.addl_product_id = 275010 THEN 0
            WHEN bs.addl_product_id = 60440 THEN 0
            WHEN bs.addl_product_id = 421531 THEN 0
            WHEN bs.addl_product_id = 123459 THEN 0.06
            WHEN bs.addl_product_id = 205461 THEN 0
            WHEN bs.addl_product_id = 313963 THEN 0.06
            WHEN bs.addl_product_id = -202941 THEN 0.05
            WHEN bs.addl_product_id = -202908 THEN 0.06
            WHEN bs.addl_product_id = 60544 THEN 0.06
            WHEN bs.addl_product_id = 132134 THEN 0.06
            WHEN bs.addl_product_id = 95430 THEN 0.06
            WHEN bs.addl_product_id = -202958 THEN 0
            WHEN bs.addl_product_id = -202929 THEN 0
            WHEN bs.addl_product_id = -202940 THEN 0
            WHEN bs.addl_product_id = -202931 THEN 0
            WHEN bs.addl_product_id = -202972 THEN 0
            WHEN bs.addl_product_id = -202906 THEN 0
            WHEN bs.addl_product_id = 59565 THEN 0
            WHEN bs.addl_product_id = -200916 THEN 0.06
            WHEN bs.addl_product_id = 225171 THEN 0.03
            WHEN bs.addl_product_id = 180632 THEN 0.04
            WHEN bs.addl_product_id = -200654 THEN 0.04
            WHEN bs.addl_product_id = 136259 THEN 0.05
            WHEN bs.addl_product_id = 19739 THEN 0.07
            WHEN bs.addl_product_id = 242062 THEN 0.07
            WHEN bs.addl_product_id = 241860 THEN 0
            WHEN bs.addl_product_id = 193561 THEN 0.08
            WHEN bs.addl_product_id = 373411 THEN 0.03
            WHEN bs.addl_product_id = 65666 THEN 0.12
            WHEN bs.addl_product_id = 270111 THEN 0
            WHEN bs.addl_product_id = 274509 THEN 0
            WHEN bs.addl_product_id = -200012 THEN 0.08
            WHEN bs.addl_product_id = -200028 THEN 0.06
            WHEN bs.addl_product_id = 46983 THEN 0
            WHEN bs.addl_product_id = 395328 THEN 0
            WHEN bs.addl_product_id = -200086 THEN 0.07
            WHEN bs.addl_product_id = 121443 THEN 0
            WHEN bs.addl_product_id = -200764 THEN 0
            WHEN bs.addl_product_id = 116701 THEN 0
            WHEN bs.addl_product_id = 116695 THEN 0
            WHEN bs.addl_product_id = -200048 THEN 0
            WHEN bs.addl_product_id = -200014 THEN 0
            WHEN bs.addl_product_id = -200050 THEN 0
            WHEN bs.addl_product_id = -200752 THEN 0
            WHEN bs.addl_product_id = -200004 THEN 0
            WHEN bs.addl_product_id = -200722 THEN 0
            WHEN bs.addl_product_id = 50311 THEN 0
            WHEN bs.addl_product_id = -200152 THEN 0
            WHEN bs.addl_product_id = 25443 THEN 0
            WHEN bs.addl_product_id = -200178 THEN 0
            WHEN bs.addl_product_id = -200898 THEN 0
            WHEN bs.addl_product_id = -200114 THEN 0
            WHEN bs.addl_product_id = -200484 THEN 0
            WHEN bs.addl_product_id = -200084 THEN 0
            WHEN bs.addl_product_id = 55680 THEN 0
            WHEN bs.addl_product_id = -200000 THEN 0
            WHEN bs.addl_product_id = -200166 THEN 0
            WHEN bs.addl_product_id = 282287 THEN 0.05
            WHEN bs.addl_product_id = 335859 THEN 0.04
            WHEN bs.addl_product_id = 282263 THEN 0.04
            WHEN bs.addl_product_id = 282279 THEN 0.04
            WHEN bs.addl_product_id = 290271 THEN 0.04
            WHEN bs.addl_product_id = 282610 THEN 0.03
            WHEN bs.addl_product_id = 287486 THEN 0.06
            WHEN bs.addl_product_id = 343750 THEN 0
            WHEN bs.addl_product_id = 280161 THEN 0.05
            WHEN bs.addl_product_id = 280159 THEN 0.06
            WHEN bs.addl_product_id = 280198 THEN 0
            WHEN bs.addl_product_id = 280195 THEN 0
            WHEN bs.addl_product_id = 280184 THEN 0
            WHEN bs.addl_product_id = 280178 THEN 0
            WHEN bs.addl_product_id = 280172 THEN 0
            WHEN bs.addl_product_id = 303714 THEN 0
            WHEN bs.addl_product_id = 280182 THEN 0.05
            WHEN bs.addl_product_id = 280160 THEN 0.07
            WHEN bs.addl_product_id = 280165 THEN 0.04
            WHEN bs.addl_product_id = 280163 THEN 0
            WHEN bs.addl_product_id = 280201 THEN 0.05
            WHEN bs.addl_product_id = 280162 THEN 0
            WHEN bs.addl_product_id = 280166 THEN 0.05
            WHEN bs.addl_product_id = 280171 THEN 0
            WHEN bs.addl_product_id = 280175 THEN 0
            WHEN bs.addl_product_id = 338423 THEN 0
            WHEN bs.addl_product_id = 395331 THEN 0
            WHEN bs.addl_product_id = 319095 THEN 0
        ELSE 0 END AS hook,
        sum(boi.quantity*foi.step_unit) as quantity,
        count(distinct bo.order_id) as ordenes,
        count(distinct bo.customer_id) as usuarios,
        CASE 
            --WHEN s.site_identifier_value = 'CMX' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.65)
            --WHEN s.site_identifier_value = 'GDL' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.65)
            --WHEN s.site_identifier_value = 'PBC' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.65)
            --WHEN s.site_identifier_value = 'MTY' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.65)
            WHEN s.site_identifier_value = 'SPO' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + coalesce(SUM(foi.total_tax_iva),0))*1.0/4.75)
            --WHEN s.site_identifier_value = 'BHZ' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + coalesce(SUM(foi.total_tax_iva),0))*1.0/4.75)
            --WHEN s.site_identifier_value = 'CWB' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + coalesce(SUM(foi.total_tax_iva),0))*1.0/4.75)
            --WHEN s.site_identifier_value = 'VCP' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + coalesce(SUM(foi.total_tax_iva),0))*1.0/4.75)
        ELSE ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3776) END AS net_gmv_usd,
        CASE 
            --WHEN s.site_identifier_value = 'CMX' then ((COALESCE(SUM(boida.adjustment_value*boipd.quantity*foi.step_unit),0))*1.0/19.65)
            --WHEN s.site_identifier_value = 'GDL' then ((COALESCE(SUM(boida.adjustment_value*boipd.quantity*foi.step_unit),0))*1.0/19.65)
            --WHEN s.site_identifier_value = 'PBC' then ((COALESCE(SUM(boida.adjustment_value*boipd.quantity*foi.step_unit),0))*1.0/19.65)
            --WHEN s.site_identifier_value = 'MTY' then ((COALESCE(SUM(boida.adjustment_value*boipd.quantity*foi.step_unit),0))*1.0/19.65)
            WHEN s.site_identifier_value = 'SPO' then ((COALESCE(SUM(boida.adjustment_value*boipd.quantity*foi.step_unit),0))*1.0/4.75)
            --WHEN s.site_identifier_value = 'BHZ' then ((COALESCE(SUM(boida.adjustment_value*boipd.quantity*foi.step_unit),0))*1.0/4.75)
            --WHEN s.site_identifier_value = 'CWB' then ((COALESCE(SUM(boida.adjustment_value*boipd.quantity*foi.step_unit),0))*1.0/4.75)
            --WHEN s.site_identifier_value = 'VCP' then ((COALESCE(SUM(boida.adjustment_value*boipd.quantity*foi.step_unit),0))*1.0/4.75)
        ELSE ((COALESCE(SUM(boida.adjustment_value*boipd.quantity*foi.step_unit),0))*1.0/3776) END AS net_discounts_usd
        
    FROM 
        postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"                  bfgi
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"            bfg     ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"             ffg     ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                        bo      ON bo.order_id = bfg.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                         s       ON bo.site_disc = s.site_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"                   boi     ON bfgi.order_item_id= boi.order_item_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                    foi     ON boi.order_item_id= foi.order_item_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                         fow     ON fow.owner_id = ffg.owner_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                         fo      ON fo.order_id = bo.order_id
        INNER JOIN ful                                                                              ON ful.order_id = bo.order_id
        LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                    fot     ON fot.fb_order_type_id=fo.fb_order_type_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"          bdoi    ON bdoi.order_item_id = boi.order_item_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = bdoi.sku_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                           bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
        left join postgres_broadleaf_federate."broadleaf.blc_category_xref"                 bcx     ON bcx.sub_category_id = bp.default_category_id and bcx.archived='N' and bcx.sndbx_tier is null and bcx.default_reference = 'true'
        left join postgres_broadleaf_federate."broadleaf.blc_category"                      bcat    ON bp.default_category_id = bcat.category_id
        left join postgres_broadleaf_federate."broadleaf.blc_category"                      bcat2   ON bcx.category_id = bcat2.category_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"          boipd   ON boipd.order_item_id=boi.order_item_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"            boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id
    WHERE 
        bo.order_status='SUBMITTED'
        AND fo.fb_order_status_id IN (1,6,7,8)
        AND (fot.name IS NULL OR fot.name <> 'REFUND')
        AND bcat2.name in ('Abarrotes','Aseo e Higiene','Bebidas','Congelados','Desechables','Lácteos & Huevos','Carne, Pollo & Pescados', 'Frutas & Verduras', 'Bebidas','Mercearia','Limpeza e Higiene','Laticínios e Ovos','Frutas e Verduras','Congelados','Descartáveis','Carnes, Aves e Peixes')
        AND (boida.adjustment_reason ILIKE '%Investment%')
    GROUP BY 1,2,3,4,5,6,7,8
    )

    SELECT 
        *,
        net_discounts_usd/net_gmv_usd AS investment_value,
        net_discounts_usd*ROUND((hook/(net_discounts_usd/net_gmv_usd)),8) AS hook_discount_usd,
        net_discounts_usd - (net_discounts_usd*ROUND((hook/(net_discounts_usd/net_gmv_usd)),8)) AS investment_discount_usd

    FROM all_data
    WHERE responsable IN ('ADDITIONAL INVESTMENT')
    """
    dataframe = read_connection_data_warehouse.runQuery(query)

In [57]:
data_df = pd.merge(city_paramss,dataframe, left_on=['card_id','city'], right_on=['addl_product_id','region_code'])#.drop(columns = ['id_tarjeta'])
data_df = data_df[['week','city','cat','subcat','name','target_gmv','target_discount','quantity','net_gmv_usd', 'net_discounts_usd','hook_discount_usd', 'investment_discount_usd']]
data_df["quantity"] = data_df["quantity"].astype(np.float64)
data_df["net_gmv_usd"] = data_df["net_gmv_usd"].astype(np.float64)
data_df["net_discounts_usd"] = data_df["net_discounts_usd"].astype(np.float64)
data_df["investment_discount_usd"] = data_df["investment_discount_usd"].astype(np.float64)
data_df["% cumplimiento GMV"] = data_df.net_gmv_usd/data_df.target_gmv
data_df["% cumplimiento Discount"] = data_df.investment_discount_usd/data_df.target_discount


data_df['Accion'] = np.where(
     data_df["% cumplimiento GMV"]>1, 
    'Apagar Descuento', 
     np.where(
        data_df["% cumplimiento Discount"]>1, 
        'Alerta!!', 
        'OK'
    )
)

data_df["% cumplimiento GMV"] = pd.Series(['{:.2%}'.format(val) for val in data_df["% cumplimiento GMV"]], index = data_df.index) 
data_df["% cumplimiento Discount"] = pd.Series(['{:.2%}'.format(val) for val in data_df["% cumplimiento Discount"]], index = data_df.index) 

In [83]:
final_df = data_df[(data_df["Accion"] != "OK")][['week','city','cat','subcat','name','% cumplimiento GMV','% cumplimiento Discount','Accion']]

In [81]:
from tabulate import tabulate

In [91]:
message ="""
Hook Stockouts en {city}: 
    
{a}
    
""".format(city="BOG", a=tabulate(final_df, showindex=False, headers=final_df.columns, tablefmt='orgtbl'))#, b=tabulate(nearStockoutList, showindex=False, headers=nearStockoutList.columns))

In [110]:
final_df.columns

Index(['week', 'city', 'cat', 'subcat', 'name', '% cumplimiento GMV',
       '% cumplimiento Discount', 'Accion'],
      dtype='object')

In [112]:
[f"""{x[4]}:
    --> % cumplimiento GMV: {x[5]} 
    --> % cumplimiento Discount: {x[6]}
    
    *ACCION:* {x[7]} 
""" for x in final_df.values]

['Cerveja Heineken 350ml:\n    --> % cumplimiento GMV: 113.93% \n    --> % cumplimiento Discount: 105.03%\n    \n    *ACCION:* Apagar Descuento \n',
 'Filé de Peito de Frango - Bello Alimentos:\n    --> % cumplimiento GMV: 81.89% \n    --> % cumplimiento Discount: 147.56%\n    \n    *ACCION:* Alerta!! \n',
 'Leite Integral Piracanjuba 1L:\n    --> % cumplimiento GMV: 196.38% \n    --> % cumplimiento Discount: 181.34%\n    \n    *ACCION:* Apagar Descuento \n',
 'Arroz Branco Camil Food Service 5kg:\n    --> % cumplimiento GMV: 59.84% \n    --> % cumplimiento Discount: 135.82%\n    \n    *ACCION:* Alerta!! \n',
 'Feijão Carioca Caldo Nobre 1kg:\n    --> % cumplimiento GMV: 89.99% \n    --> % cumplimiento Discount: 213.94%\n    \n    *ACCION:* Alerta!! \n',
 'Alho Descascado Pacote 1 kg:\n    --> % cumplimiento GMV: 66.29% \n    --> % cumplimiento Discount: 152.92%\n    \n    *ACCION:* Alerta!! \n',
 'Cebola Com Casca - Saco:\n    --> % cumplimiento GMV: 112.22% \n    --> % cumplimiento D

In [121]:
for city in ['BOG','MDE','SPO']:
    a = final_df[final_df.city == city]
    
    b = [f"""{x[4]}:
    --> % cumplimiento GMV: {x[5]} 
    --> % cumplimiento Discount: {x[6]}
    
    *ACCION:* {x[7]} 
    """ for x in a.values]
    
    b = "\n".join(b)
    
    message = f"""
    Week {final_df.week.unique()[0]} - Investment Alert for {city}:
    
        {b}
    
    """

In [122]:
print(message)


    Week 39 - Investment Alert for SPO:
    
        Cerveja Heineken 350ml:
    --> % cumplimiento GMV: 113.93% 
    --> % cumplimiento Discount: 105.03%
    
    *ACCION:* Apagar Descuento 
    
Filé de Peito de Frango - Bello Alimentos:
    --> % cumplimiento GMV: 81.89% 
    --> % cumplimiento Discount: 147.56%
    
    *ACCION:* Alerta!! 
    
Leite Integral Piracanjuba 1L:
    --> % cumplimiento GMV: 196.38% 
    --> % cumplimiento Discount: 181.34%
    
    *ACCION:* Apagar Descuento 
    
Arroz Branco Camil Food Service 5kg:
    --> % cumplimiento GMV: 59.84% 
    --> % cumplimiento Discount: 135.82%
    
    *ACCION:* Alerta!! 
    
Feijão Carioca Caldo Nobre 1kg:
    --> % cumplimiento GMV: 89.99% 
    --> % cumplimiento Discount: 213.94%
    
    *ACCION:* Alerta!! 
    
Alho Descascado Pacote 1 kg:
    --> % cumplimiento GMV: 66.29% 
    --> % cumplimiento Discount: 152.92%
    
    *ACCION:* Alerta!! 
    
Cebola Com Casca - Saco:
    --> % cumplimiento GMV: 112.22% 
    